In [441]:
import re
pattern_alph = r'[a-zA-Z]+'

In [442]:
class AVLNode:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None
        self.height = 0
    def __str__(self):
        # print data and height (in red font) of the node
        red_start = "\033[31m"
        reset_code = "\033[0m"
        return f'{self.data} {red_start}{self.height}{reset_code}'

In [443]:
class AVLTree:
    def __init__(self):
        self.root = None
    def __str__(self):
        return AVLTree._inorder(self.root, 0)
    @staticmethod
    def _preorder(node, level):
        if not node:
            return ''
        ret = f'{'\t'*level}{node}({AVLTree._get_balance(node)})\n'
        ret += AVLTree._preorder(node.right, level+1)
        ret += AVLTree._preorder(node.left, level+1)
        return ret
    @staticmethod
    def _inorder(node, level):
        if not node:
            return ''
        ret = AVLTree._inorder(node.right, level+1)
        ret += f'{'\t'*level}{node}({AVLTree._get_balance(node)})\n'
        ret += AVLTree._inorder(node.left, level+1)
        return ret
    @staticmethod
    def _postorder(node, level):
        if not node:
            return ''
        ret = AVLTree._postorder(node.right, level+1)
        ret += AVLTree._postorder(node.left, level+1)
        ret += f'{'\t'*level}{node}({AVLTree._get_balance(node)})\n'
        return ret
    @staticmethod
    def _get_height(node):
        # height of a Null node is -1
        return -1 if not node else node.height
    @staticmethod
    def _get_balance(node):
        # balance = height(left child) - height(right child)
        return 0 if not node else AVLTree._get_height(node.left) - AVLTree._get_height(node.right)
    @staticmethod
    def _rebalance(node):
        balance = AVLTree._get_balance(node)
        if balance > 1: # left heavy
            if AVLTree._get_balance(node.left) < 0:
                # Left child is right heavy => LR rotation
                node.left = AVLTree._left_rotate(node.left)
            return AVLTree._right_rotate(node)
        if balance < -1: # right heavy
            if AVLTree._get_balance(node.right) > 0:
                # right child is left heavy => RL rotation
                node.right = AVLTree._right_rotate(node.right)
            return AVLTree._left_rotate(node)
        return node
    @staticmethod
    def _left_rotate(node):
        new_root = node.right
        node.right = new_root.left
        new_root.left = node
        node.height = 1 + max(AVLTree._get_height(node.left), AVLTree._get_height(node.right))
        new_root.height = 1 + max(AVLTree._get_height(new_root.left), AVLTree._get_height(new_root.right))
        return new_root
    @staticmethod
    def _right_rotate(node):
        new_root = node.left
        node.left = new_root.right
        new_root.right = node
        node.height = 1 + max(AVLTree._get_height(node.left), AVLTree._get_height(node.right))
        new_root.height = 1 + max(AVLTree._get_height(new_root.left), AVLTree._get_height(new_root.right))
        return new_root
    @staticmethod
    def _insert(node, data): # recursive function called by insert()
        if not node:
            return AVLNode(data)
        if data == node.data: # no duplicates
            return node
        if data < node.data:
            node.left = AVLTree._insert(node.left, data)
        else:
           node.right = AVLTree._insert(node.right, data)
        node.height = 1 + max(AVLTree._get_height(node.left), AVLTree._get_height(node.right))
        return AVLTree._rebalance(node)
    @staticmethod
    def _delete(node, data): # recursive function called by delete()
        if not node:
            return None
        if node.data == data:
            if not node.right and not node.left:
                # leaf node => simply remove the node
                return None
            if not node.right:
                # no right child => promote left child
                return node.left
            if not node.left:
                # no left child => promote right child
                return node.right
            # both left and right children => promote the in-order successor
            current = node.right
            while current.left:
                current = current.left
            # copy the data from successor
            node.data = current.data
            # remove successor from right subtree
            node.right = AVLTree._delete(node.right, current.data)
        elif data < node.data:
            node.left = AVLTree._delete(node.left, data)
        else:
            node.right = AVLTree._delete(node.right, data)
        node.height = 1 + max(AVLTree._get_height(node.left), AVLTree._get_height(node.right))
        return AVLTree._rebalance(node)
    @staticmethod
    def _search(node, data): # recursive function called by search()
        if not node:
            return False
        if node.data == data:
            return True
        if data < node.data:
            return AVLTree._search(node.left, data)
        return AVLTree._search(node.right, data)
    def insert(self, data):
        self.root = AVLTree._insert(self.root, data)
    def delete(self, data):
        self.root = AVLTree._delete(self.root, data)
    def search(self, data):
        return AVLTree._search(self.root, data)
    def to_string(self, order='Inorder'):
        if order == 'Preorder':
            return AVLTree._preorder(self.root, 0)
        if order == 'Inorder':
            return AVLTree._inorder(self.root, 0)
        if order == 'Postorder':
            return AVLTree._postorder(self.root, 0)

In [444]:
def invalid_op():
    print('Please enter a valid selection')

In [445]:
Filenames = {'training': 'training.txt',
             'checking': 'testing.txt'
            }

In [446]:
def enter_text(operation):
    # Enter text from either command line or file
    def read_prompt():
        return input('Please enter text:')
    def read_file():
        file_name = input(f'Please enter file name (<Enter> = {Filenames[operation]}):')
        file_name = file_name if file_name else Filenames[operation]
        try:
            with open(file_name, 'r') as f:
                return f.read()
        except FileNotFoundError:
            print(f'Error: {file_name} not found')
        except Exception as e:
            print(f'An error occurred: {e}')
        return ''
    submenu = {'1': ('From command line', read_prompt),
               '2': ('From file', read_file),
              }
    print(f'Please enter text for {operation}')
    while True:
        for k, v in submenu.items():
            print(f'{k}: {v[0]}')
        choice = input('Please make a selection: ')
        if choice in submenu:
            txt = submenu[choice][1]()
            if txt.strip():
                return txt
            print('Please re-enter')
            continue
        invalid_op()

In [447]:
def train_dict():
    training_txt = enter_text('training')
    for word in re.findall(pattern_alph, training_txt):
        dictionary.insert(word.lower())

In [448]:
def remove_words():
    while True:
        words = input('List the words you want to remove, separated by spaces:')
        if words.strip(): # Avoid empty string
            for word in re.findall(pattern_alph, words):
                dictionary.delete(word.lower())
            break
        print('No words entered.')

In [449]:
def display_dict():
    submenu = {'1': 'Preorder',
               '2': 'Inorder',
               '3': 'Postorder'
              }
    while True:
        for k, v in submenu.items():
            print(f'{k}: {v}')
        choice = input('Please select an order (<Enter> = Inorder): ')
        choice = choice if choice else '2' # default to inorder
        if choice in submenu:
            print(dictionary.to_string(submenu[choice]))
            break
        invalid_op()

In [450]:
def check_spelling():
    typos = set()
    testing_txt = enter_text('checking')
    for word in re.findall(pattern_alph, testing_txt):
        if not dictionary.search(word.lower()):
            typos.add(word)
    print(f'Typos: {sorted(list(typos))}')

In [451]:
def print_divider():
    print('"'*80)

In [452]:
def menu():
    main_menu = {'1': ('Train dictionary', train_dict),
                 '2': ('Remove words from dictionary', remove_words),
                 '3': ('Show dictionary tree', display_dict),
                 '4': ('Check spelling', check_spelling),
                 'q': ('Quit', None)
                }
    while True:
        for k, v in main_menu.items():
            print(f'{k}: {v[0]}')
        op = input(f'Please select an option: ')
        print('\n', end='')
        if op == 'q':
            break
        main_menu.get(op, (None, invalid_op))[1]()
        print_divider()

In [ ]:
dictionary = AVLTree()
menu()

1: Train dictionary
2: Remove words from dictionary
3: Show dictionary tree
4: Check spelling
q: Quit


Please select an option:  1



Please enter text for training
1: From command line
2: From file


Please make a selection:  2
Please enter file name (<Enter> = training.txt): 


""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
1: Train dictionary
2: Remove words from dictionary
3: Show dictionary tree
4: Check spelling
q: Quit


Please select an option:  3



1: Preorder
2: Inorder
3: Postorder


Please select an order (<Enter> = Inorder):  


				words 0(0)
			with 1(0)
				uses 0(0)
		tree 3(1)
					to 0(0)
				the 1(-1)
			spell 2(-1)
				set 0(0)
	required 4(0)
					quality 0(0)
				proper 1(-1)
			order 2(-1)
				misspelled 0(0)
		methods 3(-1)
				maintains 0(0)
			inorder 1(0)
				implements 0(0)
implementation 5(0)
				identifies 0(0)
			handles 1(0)
				functionality 0(0)
		efficiently 3(1)
					duplicates 0(0)
				documentation 1(0)
					displays 0(0)
			display 2(0)
				dictionary 1(1)
					demo 0(0)
	correctly 4(0)
					code 0(0)
				checking 1(0)
					balance 0(0)
			avoid 2(-1)
				avl 0(0)
		and 3(0)
				alphabetical 1(1)
					along 0(0)
			all 2(-1)
				a 0(0)

""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
1: Train dictionary
2: Remove words from dictionary
3: Show dictionary tree
4: Check spelling
q: Quit
